In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2

# def extract_frames(video_path, output_path, frame_rate):
#     cap = cv2.VideoCapture(video_path)
#     count = 0
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         if count % frame_rate == 0:
#             frame_file = f"{output_path}/frame{count}.jpg"
#             cv2.imwrite(frame_file, frame)

#         count += 1
#     cap.release()
#     cv2.destroyAllWindows()
def extract_frames(video_path, output_path, frame_rate):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(video_fps / frame_rate)

    count = 0
    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if count == frame_index * frame_interval:
            frame_file = f"{output_path}/frame{count}.jpg"
            cv2.imwrite(frame_file, frame)
            frame_index += 1

        count += 1

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz

--2023-04-23 02:12:35--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.12.128, 2404:6800:4003:c11::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.12.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187925923 (179M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’

ssd_mobilenet_v2_co 100%[===================>] 179.22M  65.6MB/s    in 2.7s    

2023-04-23 02:12:39 (65.6 MB/s) - ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’ saved [187925923/187925923]



In [ ]:
!tar -xzf ssd_mobilenet_v2_coco_2018_03_29.tar.gz

In [ ]:
import numpy as np
import tensorflow as tf

def load_model(model_path):
    model = tf.saved_model.load(model_path)
    return model

def detect_human(frame, model):
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = model.signatures['serving_default'](input_tensor)

    # Detection scores and classes
    scores = detections['detection_scores'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int32)

    # Only keep detections with a score above a certain threshold and class 1 (person)
    indices = np.where((scores > 0.5) & (classes == 1))

    # Get the bounding boxes of detected humans
    boxes = detections['detection_boxes'][0].numpy()
    boxes = boxes[indices]

    return boxes

model_path = "ssd_mobilenet_v2_coco_2018_03_29/saved_model"
model = load_model(model_path)

In [ ]:
import os

# def process_frames(input_path, output_path, model):
#     for frame_file in os.listdir(input_path):
#         frame = cv2.imread(os.path.join(input_path, frame_file))
#         boxes = detect_human(frame, model)

#         # Draw bounding boxes around detected humans
#         for box in boxes:
#             ymin, xmin, ymax, xmax = box
#             h, w, _ = frame.shape
#             cv2.rectangle(frame, (int(xmin * w), int(ymin * h)), (int(xmax * w), int(ymax * h)), (0, 255, 0), 2)

#         # Save the processed frame
#         cv2.imwrite(os.path.join(output_path, frame_file), frame)
def process_frames(input_path, output_path, model):
    for frame_file in os.listdir(input_path):
        frame = cv2.imread(os.path.join(input_path, frame_file))
        
        if frame is not None:
            boxes = detect_human(frame, model)

            # Draw bounding boxes around detected humans
            for box in boxes:
                ymin, xmin, ymax, xmax = box
                h, w, _ = frame.shape
                cv2.rectangle(frame, (int(xmin * w), int(ymin * h)), (int(xmax * w), int(ymax * h)), (0, 255, 0), 2)

            # Save the processed frame
            cv2.imwrite(os.path.join(output_path, frame_file), frame)

In [ ]:
# def frames_to_video(input_path, output_path, fps, size):
#     out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
#     for frame_file in sorted(os.listdir(input_path), key=lambda f: int(f.split('.')[0].replace("frame", ""))):
#         frame = cv2.imread(os.path.join(input_path, frame_file))
#         out.write(frame)

#     out.release()
import re

def frames_to_video(input_path, output_path, fps, size):
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
    
    frame_files = [f for f in os.listdir(input_path) if re.match(r'frame\d+\.jpg', f)]
    
    for frame_file in sorted(frame_files, key=lambda f: int(f.split('.')[0].replace("frame", ""))):
        frame = cv2.imread(os.path.join(input_path, frame_file))
        out.write(frame)

    out.release()

In [ ]:
input_video_path = "/content/drive/MyDrive/DeepLearning/Final/Train/hugging/pexels-ksenia-chernaya-7695725-1080x1920-24fps.mp4"
output_frames_path = "/content/drive/MyDrive/DeepLearning/Final/frames"
output_processed_frames_path = "/content/drive/MyDrive/DeepLearning/Final/processed_frames"
output_video_path = "/content/drive/MyDrive/DeepLearning/Final/processed_video_hug_twopeople.mp4"

frame_rate = 4

# Extract frames from the input video
extract_frames(input_video_path, output_frames_path, frame_rate)

# Process frames (detect and draw bounding boxes around humans)
process_frames(output_frames_path, output_processed_frames_path, model)

# Get the original video's FPS and size
cap = cv2.VideoCapture(input_video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

# Convert the processed frames back to video
frames_to_video(output_processed_frames_path, output_video_path, fps, (width, height))